In [1]:
import os

os.chdir("../../../")

import asyncio
from textwrap import dedent
from typing import List, Literal, Union

from langchain.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from openai import OpenAI
from pydantic import BaseModel, Field

from src.initialization import credential_init

credential_init()

C:\Users\Ling\miniconda3\envs\aicg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage
from langchain.memory import ChatMessageHistory
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=6,
    disable_streaming=False
    # other params...
)

In [5]:
from langchain_core.tools import tool

# Step 1: Define a simple tool
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers and return the result."""
    return a + b

In [6]:
model_with_tools = model.bind_tools(tools=[add_numbers])

In [7]:
# Step 3: The conversation starts
messages = [
    HumanMessage(content="Add 3 and 7 using the tool.")
]


In [9]:
response = model_with_tools.invoke(messages)

In [10]:
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_numbers', 'arguments': '{"a": 3, "b": 7}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6a1e2a7b-233b-4adf-bbf6-ae305d4ffbb0-0', tool_calls=[{'name': 'add_numbers', 'args': {'a': 3, 'b': 7}, 'id': '3221ac46-ba56-4fcc-9b99-2ec4d1233a47', 'type': 'tool_call'}], usage_metadata={'input_tokens': 28, 'output_tokens': 7, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}})

In [15]:
tool_call = response.tool_calls[0]
print(tool_call)

{'name': 'add_numbers', 'args': {'a': 3, 'b': 7}, 'id': '3221ac46-ba56-4fcc-9b99-2ec4d1233a47', 'type': 'tool_call'}


In [16]:
result = eval(tool_call['name'])(tool_call['args'])

In [17]:
tool_msg = ToolMessage(
    content=str(result),          # usually a string or simple text
    tool_call_id=tool_call['id']    # must match the AIMessage tool_call id
)

In [18]:
messages.extend([response, tool_msg])

In [19]:
model_with_tools.invoke(messages)

AIMessage(content='The sum of 3 and 7 is 10.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--61641d68-1bcf-4f43-a903-a7b8700eee3f-0', usage_metadata={'input_tokens': 40, 'output_tokens': 14, 'total_tokens': 54, 'input_token_details': {'cache_read': 0}})